# 良く使うコマンド、パス操作

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
import pathlib
# 今のディレクトリ
here=pathlib.Path().resolve()
# 親ディレクトリ
cwd = pathlib.Path().resolve().parent
# 相対パスからつなげる
# data_path = cwd.joinpath('parent', 'other', 'child')
print(here)
print(cwd)

/workspaces/jupyter-practice
/workspaces


# Test dataset

In [3]:
import seaborn as sns
iris=sns.load_dataset('iris')


df = pd.DataFrame({ 'A' : [0, 1, 2, 3, 4, 999],
                    'B' : ["yes","no","yes","NA","no","yes"], 
                    'C' : ['dog', 'cat', 'pig', 'rat', 'horse', 'chicken'],
                    'D' : np.random.normal(loc=5, scale=3, size=6),
                    'E' : [str(i) for i in range(6)],
                    'F' : ["2012-01-01", "2012-01-03", "2012-01-06", "2012-01-09", "2012-01-21", "2012-01-31"],
                    'G' : ["2013-01-01", "2014-10-03", "2013-05-06", "2012-11-19", "2013-10-31", "2015-01-31"],
                    'H' : ["2013-01-01 01:01", "2014-10-03 01:05", "2013-05-06 01:12", "2012-11-19 06:01", "2013-10-31 11:01", "2015-01-31 01:01"],
                    'I' : ["2013-01-01 01:14", "2015-10-23 13:05", "2016-05-06 01:01", "2014-11-20 00:00", "2016-02-29 11:01", "2020-01-31 18:01"]
                    }
                  )
df

,A,B,C,D,E,F,G,H,I
0,0,yes,dog,4.124225,0,2012-01-01,2013-01-01,2013-01-01 01:01,2013-01-01 01:14
1,1,no,cat,1.677719,1,2012-01-03,2014-10-03,2014-10-03 01:05,2015-10-23 13:05
2,2,yes,pig,4.752346,2,2012-01-06,2013-05-06,2013-05-06 01:12,2016-05-06 01:01
3,3,NA,rat,-1.607465,3,2012-01-09,2012-11-19,2012-11-19 06:01,2014-11-20 00:00
4,4,no,horse,4.044656,4,2012-01-21,2013-10-31,2013-10-31 11:01,2016-02-29 11:01
5,999,yes,chicken,-0.121904,5,2012-01-31,2015-01-31,2015-01-31 01:01,2020-01-31 18:01


# 列の作成
イメージとしてはdplyrのmutate

In [4]:
# 数字をnanに変更
df.replace(999, np.nan).replace('NA', np.nan)
# pseudo encoding
df.assign(B_after = lambda df: df['B'].map({'yes': 1, 'no': 0, 'NA': np.nan}))
# if elseでencoding
df.assign(D_bin = lambda df: np.where(df['D']>=5,1,0))
# convert to categorical and encoding
# df.astype('category', categories=elem_ord, ordered=True)
# 列の追加
df.assign(total = lambda df: df.loc[:, ['A', 'D']].sum(axis=1), 
          one = 1) 
# if_else
df.assign(tmp = lambda x: np.where(x['D'] > x['D'].mean(axis=0), x['A']*x['D'] , x['A']*x['D']*0.75)) 
    
# case_when
df.assign(D_cat = lambda df: np.select([(df['D'] < 3), ((3 <= df['D']) & (df['D'] < 6)), (6 <= df['D'])], 
                                         ['small', 'medium', 'high']), 
          D_cat2 = lambda df: df.apply(lambda x: 'small' if x['D'] < 3 else 
                                       'medium' if ((x['D'] >= 3) & (x['D'] < 6)) else 
                                       'high', axis=1))

# 複雑なmutate
def temp_func(x):
    if x["B"] == "yes":
        return x["A"] * x["D"]**(-0.287)
    else:
        return 3

df.assign(temp = lambda x:x.apply(temp_func, axis = 1))

# 列の型変換
df.assign(E_cat = lambda x: pd.Categorical(x['E'], categories =['0', '1', '2', '3', '4', '5'], ordered = True), 
          A_char = lambda x: x['A'].astype('object'),
          A_char2 = lambda x: x['A'].astype('str'),
          A_float = lambda x: x['A'].astype('float32'), 
          D_int = lambda x: x['D'].astype('int'))



,A,B,C,D,E,F,G,H,I,E_cat,A_char,A_char2,A_float,D_int
0,0,yes,dog,4.124225,0,2012-01-01,2013-01-01,2013-01-01 01:01,2013-01-01 01:14,0,0,0,0.0,4
1,1,no,cat,1.677719,1,2012-01-03,2014-10-03,2014-10-03 01:05,2015-10-23 13:05,1,1,1,1.0,1
2,2,yes,pig,4.752346,2,2012-01-06,2013-05-06,2013-05-06 01:12,2016-05-06 01:01,2,2,2,2.0,4
3,3,NA,rat,-1.607465,3,2012-01-09,2012-11-19,2012-11-19 06:01,2014-11-20 00:00,3,3,3,3.0,-1
4,4,no,horse,4.044656,4,2012-01-21,2013-10-31,2013-10-31 11:01,2016-02-29 11:01,4,4,4,4.0,4
5,999,yes,chicken,-0.121904,5,2012-01-31,2015-01-31,2015-01-31 01:01,2020-01-31 18:01,5,999,999,999.0,0


# 行のfiltering
イメージとしてはdplyrのfilter

In [5]:
# 通常のFiltering
df.query("C=='horse'")
df.loc[lambda x: x['D'] < 4]
df.pipe(lambda x: x.loc[x['D'] < 4])
# 特定の文字がある行を抽出
df.pipe(lambda x:x.loc[x['C'].str.contains('hor|chi', na=False), :])


,A,B,C,D,E,F,G,H,I
4,4,no,horse,4.044656,4,2012-01-21,2013-10-31,2013-10-31 11:01,2016-02-29 11:01
5,999,yes,chicken,-0.121904,5,2012-01-31,2015-01-31,2015-01-31 01:01,2020-01-31 18:01


# 列の選択
イメージとしてはdplyrのselect

In [6]:
# select関係
df2 = df.copy().rename(columns = {"A": "good_boy", "B": "bad_boy", "C": "good_girl", "D": "bad_girl"})
df2.filter(['E', 'F'])
df2.filter(regex="$good")
df2.filter(df2.columns.str.contains("girl"))
# 複数条件での列選択
df2.loc[:, list(df2.loc[:, df2.columns.str.contains("boy")]) + ['G', 'I']]

# 列の値の種類で列を選択
df2.loc[:, df2.nunique() > 3]

# pipeを使用した方法
df2.pipe(lambda x: x.loc[:, x.columns.str.startswith("good")])
# "_"が入った正規表現
df2.pipe(lambda x: x.loc[:, x.columns.str.contains("^.*_.*$")])

# pipeを使った方法
df2.pipe(lambda x:x.loc[:, list(x.loc[:, x.columns.str.contains("boy")]) + ['G', 'I']])


,good_boy,bad_boy,G,I
0,0,yes,2013-01-01,2013-01-01 01:14
1,1,no,2014-10-03,2015-10-23 13:05
2,2,yes,2013-05-06,2016-05-06 01:01
3,3,NA,2012-11-19,2014-11-20 00:00
4,4,no,2013-10-31,2016-02-29 11:01
5,999,yes,2015-01-31,2020-01-31 18:01


In [7]:
# 時間の変換
df2=(df.assign(F_time=lambda x: pd.to_datetime(x["F"], format='%Y-%m-%d', errors = 'coerce'), 
               G_time=lambda x: pd.to_datetime(x["G"], format='%Y-%m-%d', errors = 'coerce'), 
               H_time=lambda x: pd.to_datetime(x["H"], format='%Y-%m-%d', errors = 'coerce'),
               I_time=lambda x: pd.to_datetime(x["I"], format='%Y-%m-%d', errors = 'coerce')))

# time difference
(df2.filter(regex="_time$")
 .assign(diff_FG=lambda x:(x['F_time'] - x['G_time']).dt.total_seconds(), 
         lag_F=lambda x:x['F_time'].shift(1), 
         lag_F_diff= lambda x:x['F_time']-x['F_time'].shift(1)))

,F_time,G_time,H_time,I_time,diff_FG,lag_F,lag_F_diff
0,2012-01-01,2013-01-01,NaT,NaT,-31622400.0,NaT,NaT
1,2012-01-03,2014-10-03,NaT,NaT,-86745600.0,2012-01-01,2 days
2,2012-01-06,2013-05-06,NaT,NaT,-41990400.0,2012-01-03,3 days
3,2012-01-09,2012-11-19,NaT,NaT,-27216000.0,2012-01-06,3 days
4,2012-01-21,2013-10-31,NaT,NaT,-56073600.0,2012-01-09,12 days
5,2012-01-31,2015-01-31,NaT,NaT,-94694400.0,2012-01-21,10 days


# 縦横変換

In [13]:
# wide to long
(iris.pivot(columns = 'species', values = 'petal_length'))


(pd.melt(iris, id_vars = ['species'], value_name = 'value'))

# pivot longer的な
(iris.set_index(['species', 'petal_length', 'petal_width'])
 .stack()
 .to_frame()
 .reset_index())

,species,petal_length,petal_width,level_3,0
0,setosa,1.4,0.2,sepal_length,5.1
1,setosa,1.4,0.2,sepal_width,3.5
2,setosa,1.4,0.2,sepal_length,4.9
3,setosa,1.4,0.2,sepal_width,3.0
4,setosa,1.3,0.2,sepal_length,4.7
...,...,...,...,...,...
295,virginica,5.2,2.0,sepal_width,3.0
296,virginica,5.4,2.3,sepal_length,6.2
297,virginica,5.4,2.3,sepal_width,3.4
298,virginica,5.1,1.8,sepal_length,5.9


# その他
rename等

In [ ]:
# 列名の変更
df.rename(columns = {
'A':'hoge',
'B':'fuga'}
).head()


# 条件によるImputation
(df.replace(999, np.nan).replace('NA', np.nan)
 .assign(D2 = lambda x: np.where(x['D'] > 9, np.nan, x['D']))
 .assign(D3 = lambda x: x['D2'].fillna(x['D2'].median())))

# coascelence

df.replace('NA', np.nan).assign(B_fill=lambda x: x['B'].combine_first(x['C']))


,A,B,C,D,E,F,G,H,I,B_fill
0,0,yes,dog,-0.517588,0,2012-01-01,2013-01-01,2013-01-01 01:01,2013-01-01 01:14,yes
1,1,no,cat,7.020433,1,2012-01-03,2014-10-03,2014-10-03 01:05,2015-10-23 13:05,no
2,2,yes,pig,5.757370,2,2012-01-06,2013-05-06,2013-05-06 01:12,2016-05-06 01:01,yes
3,3,NaN,rat,1.843901,3,2012-01-09,2012-11-19,2012-11-19 06:01,2014-11-20 00:00,rat
4,4,no,horse,4.067324,4,2012-01-21,2013-10-31,2013-10-31 11:01,2016-02-29 11:01,no
5,999,yes,chicken,6.890143,5,2012-01-31,2015-01-31,2015-01-31 01:01,2020-01-31 18:01,yes
